# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 37

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,29,29,2507,3,2,1,1,0,0,0.11,0.07,0.18,0.07,0.14,2.3,1.6,3.2,0.08,0.11,0.20,0.06,0.17,Matches
2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19.0,2000.0,35,35,3150,0,1,0,0,7,0,0.00,0.03,0.03,0.00,0.03,0.5,0.5,3.7,0.02,0.11,0.12,0.02,0.12,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21.0,1997.0,33,25,2202,15,3,0,0,2,0,0.61,0.12,0.74,0.61,0.74,15.1,15.1,2.5,0.62,0.10,0.72,0.62,0.72,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23.0,1996.0,29,11,1027,2,2,0,0,0,0,0.18,0.18,0.35,0.18,0.35,3.6,3.6,1.4,0.32,0.12,0.44,0.32,0.44,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32.0,1987.0,11,9,875,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, engine='python')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,37,31,3,3,82,32,50,96,70.0,40.9,29.1,0.79,W W D L W,53143,Mohamed Salah - 19,Alisson,NaN
2,Manchester City,37,25,3,9,97,35,62,78,90.0,34.0,56.1,1.52,L W W W W,54219,Raheem Sterling - 19,Ederson,NaN
3,Manchester Utd,37,17,12,8,64,36,28,63,57.9,36.2,21.7,0.59,W W D W D,72726,Marcus Rashford Anthony Martial - 17,David de Gea,NaN
4,Chelsea,37,19,6,12,67,54,13,63,66.8,37.9,28.9,0.78,W W L W L,40563,Tammy Abraham - 15,Kepa Arrizabalaga,NaN
5,Leicester City,37,18,8,11,67,39,28,62,59.5,43.5,16.0,0.43,W D L W L,32061,Jamie Vardy - 23,Kasper Schmeichel,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,242,25.0,100.0,4,69140,0,0,-4,4,45.5,348,120,0,2,0.7,3.1,0,Shkodran,2.1,17,0,47.9,306,102,False,277.2,251,98,1205,Hamstring injury - 25% chance of playing,2020-07-18T22:30:20.553834Z,51,0,0,0,69140.jpg,2.9,0,0,Mustafi,0.4,False,None,d,1,3,155.0,231,58,43,24993,121,46600,1034,0.4,8.4,Mustafi,2
2,0,4,204,100.0,100.0,4,98745,0,0,-3,3,76.9,299,93,0,2,2.9,3.4,0,Héctor,2.4,18,1,37.0,336,119,False,187.8,315,122,1156,,2019-12-09T20:00:21.228098Z,52,0,0,0,98745.jpg,2.9,0,0,Bellerín,1.0,False,None,a,1,3,103.0,272,79,44,204898,2511,157509,2484,0.5,8.5,Bellerín,2
3,2,1,325,100.0,100.0,4,111457,0,0,-3,3,172.1,216,53,0,2,3.1,3.6,1,Sead,2.6,25,0,51.2,292,96,False,263.8,265,106,1661,,2020-02-23T18:30:13.672943Z,52,0,0,0,111457.jpg,2.2,0,0,Kolasinac,0.5,False,None,a,1,3,79.0,305,98,54,64377,684,133995,301,0.5,10.4,Kolasinac,4
4,2,3,229,100.0,100.0,3,154043,0,0,-5,5,159.5,227,57,1,2,0.9,1.4,0,Ainsley,0.4,20,0,47.3,309,104,False,282.6,245,97,1292,,2019-09-22T18:00:10.824841Z,45,0,0,0,154043.jpg,2.1,1,0,Maitland-Niles,2.2,False,None,a,1,3,37.0,364,133,40,610153,914,652977,644,0.1,8.9,Maitland-Niles,4
5,0,5,305,100.0,100.0,4,39476,0,0,-2,2,36.8,363,128,1,2,0.5,1.0,0,Sokratis,0.0,25,2,58.5,268,86,False,436.2,162,60,1696,,2020-06-18T18:00:15.974146Z,48,0,0,0,39476.jpg,3.0,0,0,Papastathopoulos,1.3,False,None,a,1,3,110.0,264,75,57,181927,358,230973,528,0.0,11.9,Sokratis,6


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,37,31,3,3,82,32,50,96,70.0,40.9,29.1,0.79,W W D L W,53143,Mohamed Salah - 19,Alisson,NaN,1.105405,0.331077
2,Manchester City,37,25,3,9,97,35,62,78,90.0,34.0,56.1,1.52,L W W W W,54219,Raheem Sterling - 19,Ederson,NaN,0.918919,0.398950
3,Manchester Utd,37,17,12,8,64,36,28,63,57.9,36.2,21.7,0.59,W W D W D,72726,Marcus Rashford Anthony Martial - 17,David de Gea,NaN,0.978378,0.375920
4,Chelsea,37,19,6,12,67,54,13,63,66.8,37.9,28.9,0.78,W W L W L,40563,Tammy Abraham - 15,Kepa Arrizabalaga,NaN,1.024324,0.359039
5,Leicester City,37,18,8,11,67,39,28,62,59.5,43.5,16.0,0.43,W D L W L,32061,Jamie Vardy - 23,Kasper Schmeichel,NaN,1.175676,0.308610
6,Wolves,37,15,14,8,51,38,13,59,47.4,33.2,14.2,0.38,L L W D W,31360,Raúl Jiménez - 17,Rui Patrício,NaN,0.897297,0.407670
7,Tottenham,37,16,10,11,60,46,14,58,46.0,52.1,-6.2,-0.17,W D W W W,59384,Harry Kane - 17,Hugo Lloris,NaN,1.408108,0.244606
8,Sheffield Utd,37,14,12,11,38,36,2,54,41.3,45.7,-4.5,-0.12,D W W L L,30869,Oliver McBurnie Lys Mousset - 6,Dean Henderson,NaN,1.235135,0.290795
9,Burnley,37,15,9,13,42,48,-6,54,42.3,47.3,-5.1,-0.14,D W D D W,20260,Chris Wood - 13,Nick Pope,NaN,1.278378,0.278489


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    if df.loc[ix, 'element_type']<=2:
        # 0.24 is the probability to miss a penalty, which incurs -2 points
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] - 6 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7662084765177548

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,33.142857,232,7.0,223.746571,6.750974
281,McGovern,2.000000,11,5.5,12.597805,6.298902
618,Fernandes,12.926829,106,8.2,81.030453,6.268394
215,De Bruyne,34.117647,232,6.8,212.981479,6.242561
214,Sterling,32.131148,196,6.1,189.566972,5.899788
192,Mané,34.126984,215,6.3,193.012393,5.655712
166,Vardy,34.262295,209,6.1,192.009622,5.604109
233,Rashford,30.172414,175,5.8,167.870997,5.563724
660,Thomas,2.000000,13,6.5,10.670258,5.335129


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 1003.3
Total goals: 968
Total xG / total goals: 1.0364669421487602
